# <font color='darkblue'>__Workshop-Handout · Datenmodelle im Vergleich (SQLite · MongoDB · Neo4j · TinyFlux)__</font>

Dieses Handout fasst die wichtigsten Stärken/Schwächen, Query-Grundstrukturen sowie Beispielabfragen der vier Workshop-Teile zusammen.

---

## 1) Vergleichstabelle (kompakt)

| Aspekt | Relational (SQLite/SQL) | Dokument (MongoDB/MQL) | Graph (Neo4j/Cypher) | Zeitreihe (TinyFlux/Python) |
|---|---|---|---|---|
| **Primäres Modell** | Tabellen, Zeilen, Spalten, Schlüssel | Dokumente (JSON/BSON) mit optionalen Referenzen | Knoten & Kanten mit Labels/Properties | Punkte: `measurement` + `time` + tags + fields |
| **Abfragesprache & Grundprinzip** | **SQL**: `SELECT … FROM … WHERE … GROUP BY …` | **MQL** (JSON-Operatoren): `find()`, **Projection 1/0**, **Sort/Limit**; Aggregation-Pipeline (`$match → $project → $group → $sort`) | **Cypher**: `MATCH (a)-[r]->(b) WHERE … RETURN …` | **Python-API**: `TimeQuery`, `TagQuery`, `FieldQuery`; Ergebnis oft via **Pandas** aggregiert |
| **Übliche Indizes** | B‑Tree, FTS, Unique/Foreign Keys | Single/Compound, TTL, Text, Geospatial | Property-Index, Constraint; Traversal über Kanten | Zeit + Tag-Kombinationen implizit, Fokus auf Zeitfenster |
| **Schema/Modellierung** | Strikt/normalisiert (3NF, FK) | Flexibel: Embed vs. Reference | Beziehungen First‑Class | Append‑only Events; schmale Records, viele Punkte |
| **Stärken** | ACID, Joins, reife OLAP/Reports | Flexible Strukturen, schnelles Lesen, gute Aggregationen | Pfade, Nachbarschaften, Empfehlungen, Graph-Algorithmen | Zeitfenster, Rolling, Resampling, DAU/Retention |
| **Schwächen** | Starre Schemata, Sharding komplex | Komplexe Graph-Algorithmen umständlich | Große OLAP-Scans weniger bequem | Komplexe Joins/Transaktionen fehlen |
| **Typische Use Cases** | Transaktionen, Berichte | Inhalte/Feeds, flexible Profile | Social Graph, Routing, Knowledge Graph | Telemetrie, Metriken, IoT-Streams |

---

## 2) Query-Grundstrukturen (Cheat Sheet)

### SQL (SQLite)
```sql
-- Top-Posts nach Likes
SELECT post_id, COUNT(*) AS likes
FROM likes
GROUP BY post_id
ORDER BY likes DESC
LIMIT 5;
```

### MQL (MongoDB Query Language)
- **Filter:** JSON/BSON-Operatoren (`$eq` implizit, `$gt`, `$in`, …)
- **Projection:** Feldauswahl mit `1/0`
- **Sort/Limit:** serverseitig
- **Aggregation-Pipeline:** `$match → $project → $group → $sort → $limit`
```javascript
// Likes pro Autor (Top-5)
db.posts.aggregate([
  { $group: { _id: "$author_id", likes: { $sum: "$like_count" } } },
  { $sort: { likes: -1 } },
  { $limit: 5 },
  { $project: { _id: 0, author_id: "$_id", likes: 1 } }
]);
```

### Cypher (Neo4j)
```cypher
// Top-Posts nach Likes
MATCH (:User)-[l:LIKES]->(p:Post)
RETURN p.post_id AS post_id, COUNT(l) AS likes
ORDER BY likes DESC
LIMIT 5;
```

### TinyFlux (Python-API)
```python
from tinyflux import TagQuery, TimeQuery
from datetime import datetime, timedelta, timezone
import pandas as pd

now = datetime.now(timezone.utc)
q = (TimeQuery() >= now - timedelta(days=30)) & (TagQuery().type == "like")
rows = [ {"time": p.time, **(p.tags or {}), **(p.fields or {})} for p in db.search(q) ]
df = pd.DataFrame(rows)
top = (df.dropna(subset=["post_id"]).groupby("post_id").size()
         .reset_index(name="like_count").sort_values("like_count", ascending=False).head(5))
```

---

## 3) Q1–Q5: Wie passen die Modelle? (kurz)

- **Q1 Engagement (Posts, Comments, Likes je User):**  
  SQL: Joins/GroupBy · MQL: `$unionWith`/`$group` · Cypher: mehrere `OPTIONAL MATCH` + Summen · TinyFlux: `groupby(['user_id','type'])`

- **Q2 Top-Posts (Likes):**  
  SQL/MQL/Cypher: direkt aggregierbar · TinyFlux: Likes filtern, `groupby('post_id')`

- **Q3 Graph-Insights (Degrees, Pfad):**  
  SQL: indirekt (Join-Tabellen) · **Cypher**: natürlich (`shortestPath`) · MQL: `$lookup` + `$graphLookup` (limitiert) · TinyFlux: außerhalb des Sweet Spots

- **Q4 Trends/Rolling:**  
  SQL: Window-Functions · MQL: `$setWindowFields` (≥5.0) · Cypher+Pandas: Tagescounts → Rolling • **TinyFlux**: First‑Class (Zeitfenster + Pandas-Rolling)

- **Q5 Feed (Followings in X Tagen):**  
  SQL/MQL: Join/Lookup + Zeitfilter · Cypher: `MATCH (viewer)-[:FOLLOWS]->(a)-[:AUTHORED]->(p)` + Datum · TinyFlux: Followings-Liste + `TimeQuery` + Filter auf `user_id`

---

## 4) Setup & Struktur (1‑Seite)

```
notebooks/
  01_sqlite.ipynb
  02_mongodb.ipynb
  03_neo4j.ipynb
  04_timeseries_tinyflux.ipynb
data/
  relational/  (users.csv, posts.csv, follows.csv, likes.csv, …)
  document/    (users.jsonl, posts.jsonl, follows.jsonl)
  graph/       (nodes.csv, edges.csv)
  timeseries/  (events.csv, tinyflux_db.csv)
```

**Kernel/Umgebung:** eigene Conda-Env empfohlen (`dbworkshop`), dann Pakete:  
`pandas matplotlib python-dateutil tqdm neo4j pymongo tinyflux`

---

## 5) Troubleshooting (ultrakompakt)

- **Pandas-Darstellung crasht:** Fallback-Anzeige `show_df(df)` verwenden (im Notebook vorhanden).
- **Neo4j Bolt nicht erreichbar:** Test `Test-NetConnection 127.0.0.1 -Port 7687`; Desktop/Docker starten; URI/Passwort setzen.
- **MongoDB fehlt:** Docker Quickstart `docker run -d -p 27017:27017 mongo:6`.
- **TinyFlux leer:** `data/timeseries/events.csv` bereitstellen oder aus `relational/*.csv` ableiten lassen.

---

## 6) Stretch Goals (optional)

- **SQL:** Materialized Views / Indizes benchmarken.
- **MQL:** `$search` (Atlas Search) + Pipeline kombinieren.
- **Neo4j:** Zentralität (PageRank), Community Detection (falls APOC/Algo verfügbar).
- **TinyFlux:** Resampling (stündlich/täglich) + Anomalie‑Detektion (z. B. z‑Score).

---

*(Stand dieses Handouts: Workshop-Version 1.0)*
